 <h1> Paraphrase Detection with Neural Networks - Natural Language Understanding </h1>


CSI4106 Artificial Intelligence Project <br/>
Prepared by Abha Sharma & Rupsi Kaushik

<h2> Background </h2>

With the growing trends of virtual assistants and chatbots, Natural Language Processing (NLP) is a topic that is becoming increasingly popular in the recent years. From Google AI's Transformer-based models that consider a word's double-sided context to IBM's training data generator, today we have cutting edge approaches to solving NLP tasks.  However, even with these latest breakthroughs, NLP still faces many challenges, namely the problem of accurately deciphering what humans mean when they express something, regardless of how they express it. This problem falls under Natural Language Understanding (NLU), a subtopic of NLP that aims to increase the proficiency of intelligent systems in exhibiting real knowledge of natural language. Within this field, the task of paraphrase detection - determining whether a pair of sentences convey identical meaning - is considered to be an important one. Through the improvement of paraphrase detection, other NLP tasks that are integral to the efficiency of existing intelligent systems, such as question answering, information retrieval, and text summarization, can also be improved. For this reason, in this report, we propose to enhance the capability of neural networks in the context of paraphrase detection through the use of traditional Information Retrieval (IR) techniques as input features. 


*add and edit as you want*

<h2> Objectives </h2>

The main objective of this report is to evaluate the performance of a neural network model given different IR features. Additionally, it will take a look at how the number of features and hidden layers improve the overall performance of the model. These results will be compared among two different training sets that have been annotated for paraphrase detection. Below is the proposed architecture for our particular neural network: 

*adding the diagram & add and edit as you want* 

<h2> Datasets </h2>

We will be working with the Quora Question Pairs and Microsoft Research Paraphrase Corpus datasets for this project. You can find them in this folder labelled as 'msr_train.csv' and 'questions_train.csv'

In [147]:
#Make sure to import all these modules
import pandas as pd 
from pyemd import emd
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.wsd import lesk
from nltk import ngrams
from difflib import SequenceMatcher
from gensim.models import Word2Vec
import gensim.downloader as api
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
word_vectors = api.load("glove-wiki-gigaword-100")

In [148]:
# Taking a look at the Quora Question Pairs dataset
quora_data = pd.read_csv("questions_train.csv")
quora_data.Sentence_1 = quora_data.Sentence_1.astype(str)
quora_data.Sentence_2 = quora_data.Sentence_2.astype(str)
quora_data.head(20)

,Sentence_1,Sentence_2,is_Paraphrase
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
5,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
6,Should I buy tiago?,What keeps childern active and far from phone ...,0
7,How can I be a good geologist?,What should I do to be a great geologist?,1
8,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0
9,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0


In [149]:
#Taking a look at the Microsoft Research Paraphrase dataset 
mrp_data = pd.read_csv("msr_train.csv")
mrp_data.Sentence_1 = mrp_data.Sentence_1.astype(str)
mrp_data.Sentence_2 = mrp_data.Sentence_2.astype(str)
mrp_data.head(20)

,Sentence_1,Sentence_2,is_Paraphrase
0,"Amrozi accused his brother, whom he called ""th...","Referring to him as only ""the witness"", Amrozi...",1
1,Yucaipa owned Dominick's before selling the ch...,Yucaipa bought Dominick's in 1995 for $693 mil...,0
2,They had published an advertisement on the Int...,"On June 10, the ship's owners had published an...",1
3,"Around 0335 GMT, Tab shares were up 19 cents, ...","Tab shares jumped 20 cents, or 4.6%, to set a ...",0
4,"The stock rose $2.11, or about 11 percent, to ...",PG&E Corp. shares jumped $1.63 or 8 percent to...,1
5,Revenue in the first quarter of the year dropp...,With the scandal hanging over Stewart's compan...,1
6,"The Nasdaq had a weekly gain of 17.27, or 1.2 ...",The tech-laced Nasdaq Composite .IXIC rallied ...,0
7,The DVD-CCA then appealed to the state Supreme...,The DVD CCA appealed that decision to the U.S....,1
8,"That compared with $35.18 million, or 24 cents...",Earnings were affected by a non-recurring $8 m...,0
9,He said the foodservice pie business doesn't f...,The foodservice pie business does not fit our ...,1


<h2> Preprocessing & Transformation </h2>

In [150]:
def remove_stop_words(sentence):
    stop_words = stopwords.words('english')
    processed_sentence = [word for word in sentence if not word in stop_words]
    return processed_sentence     
def tokenize(sentence):
    tokenized_sentence = sentence.lower().split()
    return tokenized_sentence
def lemmatize(sentence):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence = [lemmatizer.lemmatize(word) for word in sentence]
    return lemmatized_sentence
def add_synonym(word):
    synonym_list = []
    for syn in wordnet.synsets(word):
        for name in syn.lemma_names():
             synonym_list.append(name.split(".")[0].replace('_',' '))
    return list(set(synonym_list))

<h2>Feature Engineering</h2>

<h3>Syntactic Similarity</h3>

<h4>Edit Distance</h4>

*Explain Edit Distance* 

In [152]:
def edit_distance(row):
    sentence_one_tokenize = tokenize(row['Sentence_1'])
    sentence_two_tokenize = tokenize(row['Sentence_2'])
    return nltk.edit_distance(sentence_one_tokenize, sentence_two_tokenize)

In [ ]:
#Applying Edit Distance to Quora sentence pairs 
quora_data['Edit_distance'] = quora_data.apply(edit_distance, axis=1)
quora_data.head(20)

In [155]:
#Applying Edit Distance to MRP Corpus sentence pairs 
mrp_data['Edit_distance'] = mrp_data.apply(edit_distance, axis=1)
mrp_data.head(20)

,Sentence_1,Sentence_2,is_Paraphrase,Edit_distance
0,"Amrozi accused his brother, whom he called ""th...","Referring to him as only ""the witness"", Amrozi...",1,11
1,Yucaipa owned Dominick's before selling the ch...,Yucaipa bought Dominick's in 1995 for $693 mil...,0,14
2,They had published an advertisement on the Int...,"On June 10, the ship's owners had published an...",1,14
3,"Around 0335 GMT, Tab shares were up 19 cents, ...","Tab shares jumped 20 cents, or 4.6%, to set a ...",0,13
4,"The stock rose $2.11, or about 11 percent, to ...",PG&E Corp. shares jumped $1.63 or 8 percent to...,1,15
5,Revenue in the first quarter of the year dropp...,With the scandal hanging over Stewart's compan...,1,8
6,"The Nasdaq had a weekly gain of 17.27, or 1.2 ...",The tech-laced Nasdaq Composite .IXIC rallied ...,0,13
7,The DVD-CCA then appealed to the state Supreme...,The DVD CCA appealed that decision to the U.S....,1,5
8,"That compared with $35.18 million, or 24 cents...",Earnings were affected by a non-recurring $8 m...,0,10
9,He said the foodservice pie business doesn't f...,The foodservice pie business does not fit our ...,1,6


<h4>Jaccard Similarity Coefficient</h4>

*Explain Jaccard*

In [ ]:
def jaccard_sim_coefficient(row):
    sentence_one_tokenize = tokenize(row['Sentence_1'])
    sentence_two_tokenize = tokenize(row['Sentence_2'])
    common_count = 0
    for word_in_one in sentence_one_tokenize:
        for word_in_two in sentence_two_tokenize:
            if word_in_one == word_in_two:
                common_count += 1
    return common_count/(len(sentence_one_tokenize) + len(sentence_two_tokenize) - common_count)
    

In [ ]:
#Applying Jaccard Similarity Coefficient to Quora sentence pairs 
quora_data['Jaccard_similarity'] = quora_data.apply(jaccard_sim_coefficient, axis=1)
quora_data.head(20)

In [ ]:
#Applying Jaccard Similarity Coefficient to MRP Corpus sentence pairs 
mrp_data['Jaccard_similarity'] = mrp_data.apply(jaccard_sim_coefficient, axis=1)
mrp_data.head(20)

<h4>Sequence Matcher</h4>

*Explain Sequence Matcher*

In [ ]:
def sequence_matcher(row):
    return SequenceMatcher(None, row['Sentence_1'], row['Sentence_2']).ratio()

In [ ]:
#Applying Sequence Matcher to Quora sentence pairs 
quora_data['Sequence_matcher'] = quora_data.apply(sequence_matcher, axis=1)
quora_data.head(20)

In [ ]:
#Applying Sequence Matcher to MRP Corpus sentence pairs 
mrp_data['Sequence_matcher'] = mrp_data.apply(sequence_matcher, axis=1)
mrp_data.head(20)

<h4>N-gram measure</h4>

*Explain N-gram measure, n=3*

In [ ]:
def ngram_measure(row):
    n = 3
    common_count = 1
    grams_sentence_one = ngrams(row['Sentence_1'].split(), n)
    grams_sentence_two = ngrams(row['Sentence_2'].split(), n)
    grams_sentence_two_total = sum(1 for x in grams_sentence_two)
    if grams_sentence_two_total == 0:
        grams_sentence_two_total = 0.001
    for gram_in_one in grams_sentence_one:
        if gram_in_one in grams_sentence_two:
            common_count += 1
    return common_count / grams_sentence_two_total

In [ ]:
#Applying N-gram Measure to Quora sentence pairs 
quora_data['N-gram_measure'] = quora_data.apply(ngram_measure, axis=1)
quora_data.head(20)

In [ ]:
#Applying N-gram Measure to MRP Corpus sentence pairs 
mrp_data['N-gram_measure'] = mrp_data.apply(ngram_measure, axis=1)
mrp_data.head(20)

<h3>Semantic Similarity</h3>

<h4> Word Mover's Distance </h4>


*Explain WMD here*

In [ ]:
def word_movers_distance(row):
    sentence_one_tokenize = tokenize(row['Sentence_1'])
    sentence_two_tokenize = tokenize(row['Sentence_2'])
    filtered_sentence_one = remove_stop_words(sentence_one_tokenize)
    filtered_sentence_two = remove_stop_words(sentence_two_tokenize)
    distance = word_vectors.wmdistance(filtered_sentence_one, filtered_sentence_two)
    return distance

In [ ]:
#Applying Word Mover's Distance to Quora sentence pairs 
quora_data['WMD_distance'] = quora_data.apply(word_movers_distance, axis=1)
quora_data.head(20)

In [ ]:
#Applying Word Mover's Distance to MRP Corpus sentence pairs 
mrp_data['WMD_distance'] = mrp_data.apply(word_movers_distance, axis=1)
mrp_data.head(20)

<h4> Pairwise Cosine Similarity </h4>

*Explain Cosine Similarity here*

In [160]:
def get_cosine_similarity(row):
    sentence_one_tokenize = tokenize(row['Sentence_1'])
    sentence_two_tokenize = tokenize(row['Sentence_2'])
    filtered_sentence_one = remove_stop_words(sentence_one_tokenize)
    filtered_sentence_two = remove_stop_words(sentence_two_tokenize)
    #lemmatize to get the root words 
    lemmatize_sentence_one = lemmatize(filtered_sentence_one)
    lemmatize_sentence_two = lemmatize(filtered_sentence_two)
    return calculate_cosine_similarity(lemmatize_sentence_one, lemmatize_sentence_two)
def calculate_cosine_similarity(sentence_one, sentence_two):
    tfidf = TfidfVectorizer(preprocessor=' '.join)
    tfidf_matrix = tfidf.fit_transform([sentence_one, sentence_two])
    similarity = cosine_similarity(tfidf_matrix)[0,1]
    return similarity

In [210]:
#Applying Cosine Similarity to MRP Corpus sentence pairs 
mrp_data['Cosine_Similarity'] = mrp_data.apply(get_cosine_similarity, axis=1)
mrp_data.head(20)

,Sentence_1,Sentence_2,is_Paraphrase,Edit_distance,Cosine_Similarity,Cosine_Classification
0,"Amrozi accused his brother, whom he called ""th...","Referring to him as only ""the witness"", Amrozi...",1,11,0.801978,1
1,Yucaipa owned Dominick's before selling the ch...,Yucaipa bought Dominick's in 1995 for $693 mil...,0,14,0.392181,0
2,They had published an advertisement on the Int...,"On June 10, the ship's owners had published an...",1,14,0.588364,1
3,"Around 0335 GMT, Tab shares were up 19 cents, ...","Tab shares jumped 20 cents, or 4.6%, to set a ...",0,13,0.448422,0
4,"The stock rose $2.11, or about 11 percent, to ...",PG&E Corp. shares jumped $1.63 or 8 percent to...,1,15,0.377462,0
5,Revenue in the first quarter of the year dropp...,With the scandal hanging over Stewart's compan...,1,8,0.776515,1
6,"The Nasdaq had a weekly gain of 17.27, or 1.2 ...",The tech-laced Nasdaq Composite .IXIC rallied ...,0,13,0.215283,0
7,The DVD-CCA then appealed to the state Supreme...,The DVD CCA appealed that decision to the U.S....,1,5,0.716812,1
8,"That compared with $35.18 million, or 24 cents...",Earnings were affected by a non-recurring $8 m...,0,10,0.237944,0
9,He said the foodservice pie business doesn't f...,The foodservice pie business does not fit our ...,1,6,0.818180,1


<h4>Named Entity Recognition Similarity</h4>

*Explain NER with jaccard*

In [ ]:
def ner_measure(row):
    ner_sentence_one=[]
    ner_sentence_two=[]
    count_common_ner = 0
    for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(row['Sentence_1']))):
        if hasattr(chunk, 'label'):
            ner_sentence_one.append(chunk)
    for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(row['Sentence_2']))):
        if hasattr(chunk, 'label'):
            ner_sentence_two.append(chunk)
    for item in ner_sentence_one:
        if item in ner_sentence_two:
            count_common_ner += 1
    return count_common_ner / (len(ner_sentence_one) + len(ner_sentence_two) + 0.001 - count_common_ner)

In [ ]:
#Applying NER Measure to MRP Corpus sentence pairs 
mrp_data['NER_similarity'] = mrp_data.apply(ner_measure, axis=1)
mrp_data.head(20)

<h4>Word Sense Disambiguation</h4>

*Explain WSD using Lesk algorithm*

In [ ]:
def wsd(row):
    sentence_one_senses = []
    sentence_two_senses = []
    common_senses = 0
    sentence_one_tokenize = tokenize(row['Sentence_1'])
    sentence_two_tokenize = tokenize(row['Sentence_2'])
    for word in sentence_one_tokenize:
        sentence_one_senses.append(lesk(row['Sentence_1'], word))
    for word in sentence_two_tokenize:
        sentence_two_senses.append(lesk(['Sentenece_2'], word))
    sentence_one_senses = (set(sentence_one_senses))
    sentence_two_senses = (set(sentence_two_senses))

    for sense in sentence_one_senses:
        if sense in sentence_two_senses:
            common_senses += 1
    return common_senses / (len(sentence_one_senses) + len(sentence_two_senses) - common_senses)

In [ ]:
#Applying WSD Measure to MRP Corpus sentence pairs 
mrp_data['WSD'] = mrp_data.apply(wsd, axis=1)
mrp_data.head(20)

<h4> Synonym Extended Cosine Similarity </h4>
Here, we will use WordNet's capabilities to extend our dictionary. We will then call our existing calculate_cosine_similarity method to get the similarity of the extended documents. 

In [131]:
def extend_sentence_synonym(row):
    sentence_one_tokenize = tokenize(row['Sentence_1'])
    sentence_two_tokenize = tokenize(row['Sentence_2'])
    filtered_sentence_one = remove_stop_words(sentence_one_tokenize)
    filtered_sentence_two = remove_stop_words(sentence_two_tokenize)
    #lemmatize to get the root words 
    lemmatize_sentence_one = lemmatize(filtered_sentence_one)
    lemmatize_sentence_two = lemmatize(filtered_sentence_two)
    # get extended synonym list    
    extended_dictionary_one = []
    extended_dictionary_two = []
    for one, two in zip(lemmatize_sentence_one, lemmatize_sentence_two):
        synonym_one = add_synonym(one)
        synonym_two = add_synonym(two)
        if(synonym_one):
            extended_dictionary_one += synonym_one
        if(synonym_two):
            extended_dictionary_two += synonym_two
    lemmatize_sentence_one += extended_dictionary_one
    lemmatize_sentence_two += extended_dictionary_two
    
    #calculate similarity based on the extended list 
    similarity = calculate_cosine_similarity(lemmatize_sentence_one, lemmatize_sentence_two)
    return similarity

In [133]:
mrp_data['Synonym_Cosine'] = mrp_data.apply(extend_sentence_synonym, axis = 1)
mrp_data.head(20)

,Sentence_1,Sentence_2,is_Paraphrase,Synonym_Cosine
0,"Amrozi accused his brother, whom he called ""th...","Referring to him as only ""the witness"", Amrozi...",1,0.392325
1,Yucaipa owned Dominick's before selling the ch...,Yucaipa bought Dominick's in 1995 for $693 mil...,0,0.089886
2,They had published an advertisement on the Int...,"On June 10, the ship's owners had published an...",1,0.693234
3,"Around 0335 GMT, Tab shares were up 19 cents, ...","Tab shares jumped 20 cents, or 4.6%, to set a ...",0,0.487594
4,"The stock rose $2.11, or about 11 percent, to ...",PG&E Corp. shares jumped $1.63 or 8 percent to...,1,0.473946
5,Revenue in the first quarter of the year dropp...,With the scandal hanging over Stewart's compan...,1,0.829344
6,"The Nasdaq had a weekly gain of 17.27, or 1.2 ...",The tech-laced Nasdaq Composite .IXIC rallied ...,0,0.167325
7,The DVD-CCA then appealed to the state Supreme...,The DVD CCA appealed that decision to the U.S....,1,0.191799
8,"That compared with $35.18 million, or 24 cents...",Earnings were affected by a non-recurring $8 m...,0,0.045961
9,He said the foodservice pie business doesn't f...,The foodservice pie business does not fit our ...,1,0.831294


<h4> Hypernym Extended Similarity </h4>
Checking if words are near by in hypernym hierarchy  <br/>
Wordnet hierarchy augmented with probabilities  <br />
Then calculate shortest path based on similarity


<h2> Train Set & Test Set </h2>

For training with less bias, we want to maintain a reasonably equal class distribution. We want to make sure that we have a balanced class of non-paraphrase, as well as paraphrase pairs of sentences. If this is not possible, as with most real-life scenarios, we need to think of a way to adjust our metric for evaluation in order to accomodate for this imbalance. 

In [241]:
# 70 - 30 split
#print(mrp_data.index) num of rows * 0.7 = approx 1605
mrp_data_train = mrp_data[:1605]
mrp_data_test = mrp_data[1605:]

<h2> Model </h2>

<h3> Baseline Model </h3>

In [186]:
def apply_cosine_classification(row):
    if (row['Cosine_Similarity'] > 0.5):
        classification = 1
    else:
        classification = 0
    return classification

In [187]:
mrp_data['Cosine_Classification'] = mrp_data.apply(apply_cosine_classification, axis=1)
mrp_data.head(20)

,Sentence_1,Sentence_2,is_Paraphrase,Edit_distance,Cosine_Similarity,Cosine_Classification
0,"Amrozi accused his brother, whom he called ""th...","Referring to him as only ""the witness"", Amrozi...",1,11,0.801978,1
1,Yucaipa owned Dominick's before selling the ch...,Yucaipa bought Dominick's in 1995 for $693 mil...,0,14,0.392181,0
2,They had published an advertisement on the Int...,"On June 10, the ship's owners had published an...",1,14,0.588364,1
3,"Around 0335 GMT, Tab shares were up 19 cents, ...","Tab shares jumped 20 cents, or 4.6%, to set a ...",0,13,0.448422,0
4,"The stock rose $2.11, or about 11 percent, to ...",PG&E Corp. shares jumped $1.63 or 8 percent to...,1,15,0.377462,0
5,Revenue in the first quarter of the year dropp...,With the scandal hanging over Stewart's compan...,1,8,0.776515,1
6,"The Nasdaq had a weekly gain of 17.27, or 1.2 ...",The tech-laced Nasdaq Composite .IXIC rallied ...,0,13,0.215283,0
7,The DVD-CCA then appealed to the state Supreme...,The DVD CCA appealed that decision to the U.S....,1,5,0.716812,1
8,"That compared with $35.18 million, or 24 cents...",Earnings were affected by a non-recurring $8 m...,0,10,0.237944,0
9,He said the foodservice pie business doesn't f...,The foodservice pie business does not fit our ...,1,6,0.818180,1


<h3> Multi-layer Perceptron </h3>

<h2> Results & Evaluation </h2>

In [229]:
def calculate_accuracy(model, actual_tags):
    correct = 0
    total = 0
    for prediction, actual in zip(model, actual_tags):
        total += 1
        if prediction == actual:
            correct += 1 
    accuracy = correct / total 
    return '{0:.1%}'.format(accuracy)
def build_confusion_matrix(actual_tags, model, classOfInterest):
    confusion_matrix = {}
    truePositives = len([p for p, a in zip(model, actual_tags) if p == a and p == classOfInterest])
    trueNegatives = len([p for p, a in zip(model, actual_tags) if p == a and p != classOfInterest])
    falsePositives = len([p for p, a in zip(model, actual_tags) if p != a and p == classOfInterest])
    falseNegatives = len([p for p, a in zip(model, actual_tags) if p != a and p != classOfInterest])
    confusion_matrix["tp"] = truePositives
    confusion_matrix["tn"] = trueNegatives
    confusion_matrix["fp"] = falsePositives 
    confusion_matrix["fn"] = falseNegatives
    return confusion_matrix
def calculate_recall(model, actual_tags, classOfInterest):
    matrix = build_confusion_matrix(model, actual_tags, classOfInterest)
    recall = matrix["tp"] / ( matrix["tp"] + matrix ["fn"])
    return '{0:.1%}'.format(recall)
def calculate_precision(model, actual_tags, classOfInterest):
    matrix = build_confusion_matrix(model, actual_tags, classOfInterest)
    precision = matrix["tp"]/ (matrix["tp"] + matrix["fp"])
    return '{0:.1%}'.format(precision)

In [226]:
baseline_accuracy = calculate_accuracy(mrp_data['Cosine_Classification'], mrp_data['is_Paraphrase'])
baseline_recall = calculate_recall(mrp_data['Cosine_Classification'], mrp_data['is_Paraphrase'], 1)
baseline_precision = calculate_precision(mrp_data['Cosine_Classification'], mrp_data['is_Paraphrase'], 1)

In [228]:
baseline_evaluation_summary = {"Model": ['Baseline'],
                   "Accuracy":[(baseline_accuracy)], 
                   "Recall":[baseline_recall], 
                   "Precision":[baseline_precision]}
results_df = pd.DataFrame(baseline_evaluation_summary)
print(results_df)

      Model Accuracy Recall Precision
0  Baseline    66.7%  81.3%     65.3%


<h2> References </h2>